In [22]:
from dataclasses import dataclass
from enum import Enum, IntEnum
from typing import List, Optional
import random

ModuleNotFoundError: No module named 'PIL'

2

In [44]:


class Inventory(IntEnum):
    LEAF = 1

@dataclass
class Position:
    x: int
    y: int
    
    @staticmethod
    def random(max_x: int, max_y: int) -> Position:
        x = random.randrange(max_x)
        y = random.randrange(max_y)
        return Position(x,y)

@dataclass
class Size:
    width: int
    height: int

class Caste(IntEnum):
    WORKER = 1
    SOLDIER = 2
    QUEEN = 3

    @classmethod
    def random(cls) -> Caste:
        int_value = random.randrange(len(Caste))
        return scls(int_value)

@dataclass
class Ant():
    caste: Caste
    position: Position
    age: int
    health: int
    food: int
    water: int
    inventory: List[Inventory]
    history: List[str]


class TerraineItem(Enum):
    GRASS = 0
    DIRT = 1
    WATER = 2
    TREE = 3
    NEST = 4

    def text(self) -> str:
        match self:
            case TerraineItem.GRASS:
                return "G"
            case TerraineItem.DIRT:
                return "D"
            case TerraineItem.WATER:
                return "W"
            case TerraineItem.TREE:
                return "T"
            case TerraineItem.NEXT:
                return "N"



@dataclass
class Map():
    size: Size
    terraine: List[TerraineItem]

    def set_item(self, pos:Position, item:TerraineItem):
        index = self.size.width * pos.y + pos.x
        self.terraine[index] = item

    def get_item(self, pos: Position) -> TerraineItem:
        index = self.size.width * pos.y + pos.x
        return self.terraine[index]


@dataclass
class World():
    ants: List[Ant]
    map: Map

    def display(self) -> str:
        text_map = ""
        size = self.map.size
        for y in range(size.width):
            for x in range(size.height):
                item = self.map.get_item(Position(x,y))
                text_map += item.text() + "  "
            text_map += "\n"
            
        return text_map

def creat_world() -> World:
    map_size = Size(20, 20)

    ants = []
    for _ in range(5):
        pos = Position.random(map_size.width, map_size.height)
        ant = Ant(Caste.WORKER, pos, 0, 100, 100, 100, [], [])
        ants.append(ant)

    terraine = [TerraineItem.GRASS for _ in range(map_size.width*map_size.height)]
    map = Map(map_size, terraine)

    return World(ants, map)
    




In [45]:
world = creat_world()
print(world.display())m,

G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  G  
G  G  G  G  G  G  G  G  

In [36]:
Size(0,0).height

0